<a href="https://colab.research.google.com/github/Ajay-user/DataScience/blob/master/Notes/Introduction_to_the_Keras_Tuner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://www.tensorflow.org/tutorials/keras/keras_tuner#download_and_prepare_the_dataset


Using the Keras Tuner to perform hypertuning for an image classification application.

In [1]:
pip install keras-tuner

     |████████████████████████████████| 97 kB 6.9 MB/s 


In [2]:
import tensorflow as tf
import keras_tuner as kt

The Keras Tuner is a library that helps you pick the optimal set of hyperparameters for your TensorFlow program. The process of selecting the right set of hyperparameters for your machine learning (ML) application is called hyperparameter tuning or hypertuning.

**Hyperparameters are of two types:**

* Model hyperparameters which influence model selection such as the number and width of hidden layers
* Algorithm hyperparameters which influence the speed and quality of the learning algorithm such as the learning rate for Stochastic Gradient Descent (SGD) and the number of nearest neighbors for a k Nearest Neighbors (KNN) classifier

## Download and prepare the dataset

In [3]:
(train_images, train_labels),(test_images, test_labels) = tf.keras.datasets.fashion_mnist.load_data()

4431872/4422102 [==============================] - 0s 0us/step


In [4]:
print('Shape of training images',train_images.shape)
print('Shape of training labels',train_labels.shape)
print('Shape of testing images',test_images.shape)
print('Shape of testing labels',test_labels.shape)

Shape of training images (60000, 28, 28)
Shape of training labels (60000,)
Shape of testing images (10000, 28, 28)
Shape of testing labels (10000,)


## Normalize pixel values between 0 and 1

In [5]:
train_images_norm = train_images.astype('float32')/255
test_images_norm = test_images.astype('float32')/255

## Model building

When you build a model for hypertuning, you also define the hyperparameter search space in addition to the model architecture. The model you set up for hypertuning is called a hypermodel.

You can define a hypermodel through two approaches:

* By using a model builder function
* By subclassing the HyperModel class of the Keras Tuner API

We can also use two pre-defined HyperModel classes - `HyperXception` and `HyperResNet` for computer vision applications.

In [6]:
def model_builder(hp):
  model = tf.keras.Sequential()
  model.add(tf.keras.layers.Flatten(input_shape=(28,28)))

  # tune the number of units in the dense layer
  # hp.int(units , ..range..) -- sample an integer from the range
  hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
  model.add(tf.keras.layers.Dense(units=hp_units, activation='relu'))
  model.add(tf.keras.layers.Dense(10))

  # this fn returns a compiled model
  hp_lr = hp.Choice('learning_rate', values=[1e-2,1e-3,1e-4])
  model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=hp_lr),
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])
  
  return model

Instantiate the tuner to perform the hypertuning. The Keras Tuner has four tuners available - RandomSearch, Hyperband, BayesianOptimization, and Sklearn. 

To instantiate the Hyperband tuner, you must specify the hypermodel, the objective to optimize and the maximum number of epochs to train (max_epochs).

In [7]:
tuner = kt.Hyperband(model_builder, objective='val_accuracy', max_epochs=10, factor=3, directory='my_dir', project_name='intro_to_kt')

The Hyperband tuning algorithm uses adaptive resource allocation and early-stopping to quickly converge on a high-performing model. This is done using a sports championship style bracket. The algorithm trains a large number of models for a few epochs and carries forward only the top-performing half of models to the next round. Hyperband determines the number of models to train in a bracket by computing 1 + logfactor(max_epochs) and rounding it up to the nearest integer.

In [8]:
early_stopping_cb = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=1)

Run the hyperparameter search. The arguments for the search method are the same as those used for tf.keras.model.fit in addition to the callback above.

In [9]:
tuner.search(train_images_norm, train_labels, epochs=50, validation_split=0.2, callbacks=[early_stopping_cb])

Trial 30 Complete [00h 00m 21s]
val_accuracy: 0.8460833430290222

Best val_accuracy So Far: 0.8829166889190674
Total elapsed time: 00h 10m 47s
INFO:tensorflow:Oracle triggered exit


In [10]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]


print('Best Hyperparameters found')
print('No: units in first hidden layer',best_hps.get('units'))
print('Learning rate for the optimizer',best_hps.get('learning_rate'))

Best Hyperparameters found
No: units in first hidden layer 320
Learning rate for the optimizer 0.001


## Train the model
Find the optimal number of epochs to train the model with the hyperparameters obtained from the search.

In [12]:
model = tuner.hypermodel.build(best_hps)
history = model.fit(train_images_norm, train_labels, validation_split=0.2, epochs=50, callbacks=[early_stopping_cb])

Epoch 1/50
1500/1500 [==============================] - 6s 4ms/step - loss: 0.5005 - accuracy: 0.8217 - val_loss: 0.4155 - val_accuracy: 0.8472
Epoch 2/50
1500/1500 [==============================] - 5s 4ms/step - loss: 0.3709 - accuracy: 0.8643 - val_loss: 0.3734 - val_accuracy: 0.8658
Epoch 3/50
1500/1500 [==============================] - 6s 4ms/step - loss: 0.3321 - accuracy: 0.8786 - val_loss: 0.3579 - val_accuracy: 0.8721
Epoch 4/50
1500/1500 [==============================] - 6s 4ms/step - loss: 0.3097 - accuracy: 0.8854 - val_loss: 0.3574 - val_accuracy: 0.8758
Epoch 5/50
1500/1500 [==============================] - 5s 4ms/step - loss: 0.2901 - accuracy: 0.8932 - val_loss: 0.3351 - val_accuracy: 0.8792
Epoch 6/50
1500/1500 [==============================] - 6s 4ms/step - loss: 0.2753 - accuracy: 0.8972 - val_loss: 0.3183 - val_accuracy: 0.8840
Epoch 7/50
1500/1500 [==============================] - 5s 4ms/step - loss: 0.2609 - accuracy: 0.9019 - val_loss: 0.3126 - val_accuracy:

In [15]:
val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch))+1

In [16]:
print('best epoch',best_epoch)

best epoch 7


Re-instantiate the hypermodel and train it with the optimal number of epochs.

In [17]:
hyper_model = tuner.hypermodel.build(best_hps)
history_hm = hyper_model.fit(train_images_norm, train_labels, validation_split=0.2, epochs=best_epoch)

Epoch 1/7
1500/1500 [==============================] - 6s 4ms/step - loss: 0.4967 - accuracy: 0.8244 - val_loss: 0.4107 - val_accuracy: 0.8486
Epoch 2/7
1500/1500 [==============================] - 6s 4ms/step - loss: 0.3731 - accuracy: 0.8641 - val_loss: 0.3661 - val_accuracy: 0.8686
Epoch 3/7
1500/1500 [==============================] - 6s 4ms/step - loss: 0.3342 - accuracy: 0.8775 - val_loss: 0.3720 - val_accuracy: 0.8694
Epoch 4/7
1500/1500 [==============================] - 6s 4ms/step - loss: 0.3079 - accuracy: 0.8860 - val_loss: 0.3553 - val_accuracy: 0.8742
Epoch 5/7
1500/1500 [==============================] - 5s 4ms/step - loss: 0.2896 - accuracy: 0.8916 - val_loss: 0.3231 - val_accuracy: 0.8851
Epoch 6/7
1500/1500 [==============================] - 5s 4ms/step - loss: 0.2729 - accuracy: 0.8973 - val_loss: 0.3556 - val_accuracy: 0.8687
Epoch 7/7
1500/1500 [==============================] - 6s 4ms/step - loss: 0.2612 - accuracy: 0.9021 - val_loss: 0.3234 - val_accuracy: 0.8852

Evaluate the hypermodel on the test data.

In [18]:
loss, accuracy = hyper_model.evaluate(test_images_norm, test_labels)
print('Loss', loss)
print('Accuracy',accuracy)

313/313 [==============================] - 1s 2ms/step - loss: 0.3523 - accuracy: 0.8754
Loss 0.3523252010345459
Accuracy 0.8754000067710876
